This part of the code generates a new csv containing the Lab schedule without any collision.

In [3]:
!pip install python-constraint



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import csv
from constraint import Problem, AllDifferentConstraint

def get_lab_list(file_path, row_name):
    lab_list = []
    with open(file_path, 'r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        if row_name not in reader.fieldnames:
            return None  # The specified row name is not found in the CSV file
        for row in reader:
            if row[row_name] not in lab_list:
                lab_list.append(row[row_name])
    return lab_list

# Read the class schedule from the CSV file
class_schedule = []
with open('ProjectSchedule.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        class_schedule.append(row)

# Get the list of labs from the CSV file
file_path = 'ProjectSchedule.csv'
row_name = 'ROOM'
labs = get_lab_list(file_path, row_name)

# Create a CSP problem instance
problem = Problem()

# Add variables to the problem, one for each class
for i, cls in enumerate(class_schedule):
    problem.addVariable(i, labs)


# Add constraints to the problem
for i in range(len(class_schedule)):
    for j in range(i+1, len(class_schedule)):
        cls1 = class_schedule[i]
        cls2 = class_schedule[j]
        if cls1['DAYS'] == cls2['DAYS'] and cls1['TIME'] == cls2['TIME']:
            # Ensure that no two classes are scheduled in the same room at the same time
            problem.addConstraint(AllDifferentConstraint(), [i, j])
            # Ensure that the same faculty member is not scheduled to teach two different classes at the same time
            faculty1 = cls1['FACULTY'].replace('"', '').replace(',', '').strip()
            faculty2 = cls2['FACULTY'].replace('"', '').replace(',', '').strip()
            if faculty1 == faculty2:
                problem.addConstraint(lambda x, y: x != y, [i, j])



# Find a solution to the problem
solution = problem.getSolution()



# Write the solution to a new CSV file
with open('NewLabSchedule5.csv', 'w', newline='') as csvfile:
    fieldnames = ['Days', 'Time', 'Faculty', 'Room']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i, lab in solution.items():
        cls = class_schedule[i]
        writer.writerow({'Days': cls['DAYS'], 'Time': cls['TIME'], 'Faculty': cls['FACULTY'], 'Room': cls['ROOM']})



In [3]:
import csv
from constraint import Problem, AllDifferentConstraint
from collections import defaultdict

def get_lab_list(file_path, row_name):
    lab_list = []
    with open(file_path, 'r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        if row_name not in reader.fieldnames:
            return None  # The specified row name is not found in the CSV file
        for row in reader:
            if row[row_name] not in lab_list:
                lab_list.append(row[row_name])
    return lab_list

# Read the class schedule from the CSV file
class_schedule = []
with open('ProjectSchedule.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        class_schedule.append(row)

# Create a dictionary to store the classes by (DAYS, TIME)
classes_by_day_time = defaultdict(list)

# Populate the dictionary with data from the class schedule
for cls in class_schedule:
    day_time = (cls['DAYS'], cls['TIME'])
    classes_by_day_time[day_time].append(cls)

# Get the list of labs from the CSV file
file_path = 'ProjectSchedule.csv'
row_name = 'ROOM'
labs = get_lab_list(file_path, row_name)

# Create a CSP problem instance
problem = Problem()

# Add variables to the problem, one for each class
for i, cls in enumerate(class_schedule):
    problem.addVariable(i, labs)

# Add constraints to the problem
for i in range(len(class_schedule)):
    for j in range(i+1, len(class_schedule)):
        cls1 = class_schedule[i]
        cls2 = class_schedule[j]
        if cls1['DAYS'] == cls2['DAYS'] and cls1['TIME'] == cls2['TIME']:
            # Ensure that no two classes are scheduled in the same room at the same time
            problem.addConstraint(AllDifferentConstraint(), [i, j])
            # Ensure that the same faculty member is not scheduled to teach two different classes at the same time
            faculty1 = cls1['FACULTY'].replace('"', '').replace(',', '').strip()
            faculty2 = cls2['FACULTY'].replace('"', '').replace(',', '').strip()
            if faculty1 == faculty2:
                problem.addConstraint(lambda x, y: x != y, [i, j])

# Find a solution to the problem
solution = problem.getSolution()

# Write the solution to a new CSV file
with open('NewLabSchedule6.csv', 'w', newline='') as csvfile:
    fieldnames = ['Days', 'Time', 'Faculty', 'Room']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i, lab in solution.items():
        cls = class_schedule[i]
        writer.writerow({'Days': cls['DAYS'], 'Time': cls['TIME'], 'Faculty': cls['FACULTY'], 'Room': cls['ROOM']})


In [6]:
!pip install pandas


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


Below we prompt the user to choose to download a csv file based on the member of facualty they want or the data on the labs/room for a specific room.

Below I just modified the code to allow either mixture of small or capital letter to be normalized and return the results.

In [4]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('NewLabSchedule6.csv')

while True:
    # Ask the user to specify a group
    group_by = input('Do you want to group by Room or Faculty? ')
    group_value = input(f'Enter the {group_by} you want to generate a CSV file for: ')

    # Normalize the group value entered by the user
    normalized_group_value = ' '.join(group_value.lower().split())

    # Check if the specified group exists in the data
    mask = df[group_by].str.lower().str.split().str.join(' ') == normalized_group_value
    if not any(mask):
        print(f'Error: {group_by} {group_value} not found in the data')
        retry = input('Do you want to try again? (y/n) ')
        if retry.lower() != 'y':
            break
    else:
        # Filter the data to include only the specified group
        filtered_df = df[mask]

        # Get the original name of the group value from the data
        original_group_value = filtered_df.iloc[0][group_by]

        # Write the filtered data to a new CSV file
        file_name = f'{original_group_value}.csv'
        filtered_df.to_csv(file_name, index=False)
        print(f'The data for {group_by} {original_group_value} has been written to {file_name}')
        break


The data for Room SS14 has been written to SS14.csv
